# Imports

In [1]:
# Built-In
from importlib import reload

# 3rd party
import pandas
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from keras import layers
from keras.models import Model, Sequential

# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

# In-House


2022-09-06 18:47:42.808264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 18:47:42.808304: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Intra
Separated because we need to reload often

In [2]:
import helpers
from helpers import get_damage_profile_composition

reload(helpers)

<module 'helpers' from '/home/beaujb/github/faker-classifier/helpers.py'>

# Globals

In [3]:
RANDOM_STATE = 30

# Load Data

In [4]:
file_path: str = "./merged9.csv"
# Load csv
print(f'Loading "{file_path}"')
data: pandas.DataFrame = pandas.read_csv(file_path)
print(f"Finish loading.")
print(data.shape)
data.head()

Loading "./merged9.csv"
Finish loading.
(987, 65)


,Date,Tournament,W/L,Side,Vs_Team,Game_Length,Champion,Vs_Champion,Kills,Deaths,...,teammate_top_ign,teammate_jungle_ign,teammate_mid_ign,teammate_adc_ign,teammate_support_ign,opponent_top_ign,opponent_jungle_ign,opponent_mid_ign,opponent_adc_ign,opponent_support_ign
0,2013-04-06 10:30:00,"Korea,Champions 2013 Spring",Win,Red,CJ Entus Blaze,35:42,Nidalee,Kha'Zix,6,0,...,Impact,bengi,Faker,Piglet,PoohManDu,Flame,Helios (Shin Dong-jin),Ambition,Cpt Jack,Lustboy
1,2013-04-06 10:30:01,"Korea,Champions 2013 Spring",Win,Blue,CJ Entus Blaze,35:23,Karthus,Diana,12,2,...,Impact,bengi,Faker,Piglet,PoohManDu,Flame,Helios (Shin Dong-jin),Ambition,Cpt Jack,Lustboy
2,2013-04-12 09:30:00,"Korea,Champions 2013 Spring",Loss,Red,NaJin Sword,28:49,Lux,Twisted Fate,2,6,...,Impact,bengi,Faker,Piglet,PoohManDu,MakNooN,watch,SSONG,PraY,Cain
3,2013-04-12 09:30:01,"Korea,Champions 2013 Spring",Win,Blue,NaJin Sword,33:24,Twisted Fate,Zilean,7,3,...,Impact,bengi,Faker,Piglet,PoohManDu,MakNooN,watch,SSONG,PraY,Cain
4,2013-04-17 09:30:00,"Korea,Champions 2013 Spring",Win,Blue,MVP Blue,38:43,Jayce,Kha'Zix,6,3,...,Impact,bengi,Faker,Piglet,PoohManDu,Cheonju,ChuNyang,Easyhoon,Deft,FLahm


In [5]:
data.columns

Index(['Date', 'Tournament', 'W/L', 'Side', 'Vs_Team', 'Game_Length',
       'Champion', 'Vs_Champion', 'Kills', 'Deaths', 'Assists', 'KDA',
       'Spells', 'CS', 'Delta_CS', 'Items', 'Date_parsed',
       'tournaments_played', 'date', 'blue_side_team', 'red_side_team',
       'winner', 'blue_side_bans', 'red_side_bans', 'blue_side_picks',
       'red_side_picks', 'blue_side_roster', 'red_side_roster',
       'tournament_curr_total_games', 'tournament_curr_wins',
       'tournament_curr_losses', 'tournament_curr_win_percentage',
       'tournament_type', 'teams_region', 'teammate_role_top',
       'teammate_role_jungle', 'teammate_role_mid', 'teammate_role_adc',
       'teammate_role_support', 'enemy_role_top', 'enemy_role_jungle',
       'enemy_role_mid', 'enemy_role_adc', 'enemy_role_support', 'bans',
       'teammate_top_champion', 'teammate_jungle_champion',
       'teammate_mid_champion', 'teammate_adc_champion',
       'teammate_support_champion', 'opponent_top_champion',
      

# Feature Engineer

## add "Team's Damage Composition"

In [6]:
data[["blue_physical_damage_perc",
      "blue_magic_damage_perc",
      "blue_true_damage_perc",
      "red_physical_damage_perc",
      "red_magic_damage_perc",
      "red_true_damage_perc"]] = data.apply(get_damage_profile_composition, axis=1, result_type="expand")

# Preprocess

In [7]:
cols_to_use = [
    "Side",
    "tournament_curr_win_percentage",
    "teams_region",
    "teammate_role_top",
    "teammate_role_jungle",
    "teammate_role_mid",
    "teammate_role_adc",
    "teammate_role_support",
    "enemy_role_top",
    "enemy_role_jungle",
    "enemy_role_mid",
    "enemy_role_adc",
    "enemy_role_support",
    "tournament_type",
    "blue_physical_damage_perc",
    "blue_magic_damage_perc",
    "blue_true_damage_perc",
    "red_physical_damage_perc",
    "red_magic_damage_perc",
    "red_true_damage_perc",
]

X = data[cols_to_use]

## transform target column to discrete values

In [8]:
y = data["W/L"]
def transform_game_result(game_result):
    if game_result.lower() == "win":
        return 1
    elif game_result.lower() == "loss":
        return 0
    else:
        raise ValueError("\"W/L\" column has invalid values")
y = y.transform(transform_game_result)

## create training, dev, and test set

In [9]:
# Train Set is 60% of entire data set
# Dev Set is 20% of entire data set
# Test Set is 20% of entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=RANDOM_STATE)

X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, train_size=0.75, test_size=0.25, random_state=RANDOM_STATE)

## get categorical cols

In [10]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
print(f"categorical_cols = {categorical_cols}\n")

categorical_cols = ['Side', 'teams_region', 'teammate_role_top', 'teammate_role_jungle', 'teammate_role_mid', 'teammate_role_adc', 'teammate_role_support', 'enemy_role_top', 'enemy_role_jungle', 'enemy_role_mid', 'enemy_role_adc', 'enemy_role_support', 'tournament_type']



## get numerical cols

In [11]:
# Get numerical cols
numerical_cols = [col for col in X_train.columns
                  if X_train[col].dtype in ["int64", "float64"]]
print(f"numerical_cols = {numerical_cols}")

numerical_cols = ['tournament_curr_win_percentage', 'blue_physical_damage_perc', 'blue_magic_damage_perc', 'blue_true_damage_perc', 'red_physical_damage_perc', 'red_magic_damage_perc', 'red_true_damage_perc']


In [12]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Build NN

In [13]:
# def get_model(hidden_layer_dim, meta):
#     n_features_in_ = meta["n_features_in_"]
#     X_shape_ = meta["X_shape_"]
#     n_classes_ = meta["n_classes_"]
#     print(f"X_shape={X_shape_}")
#     model = keras.Sequential(
#         [
#             # layers.Dense(512, activation="relu", input_shape=[X_shape_[1:]]),
#             layers.Dropout(rate=0.5),
#             layers.BatchNormalization(),
#             layers.Dense(512, activation="relu"),
#             layers.Dropout(rate=0.5),
#             layers.BatchNormalization(),
#             # layers.Dense(512, activation='relu'),
#             # layers.Dropout(rate=0.1),
#             # layers.BatchNormalization(),
#             # layers.Dense(512, activation='relu'),
#             # layers.Dropout(rate=0.1),
#             # layers.BatchNormalization(),
#             layers.Dense(1, activation="sigmoid"),
#         ]
#     )

#     model.compile(
#         optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"]
#     )

#     return model


In [14]:
def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.Sequential(
        [
            layers.Dense(n_features_in_, input_shape=X_shape_[1:]),
            layers.Dropout(rate=0.5),
            layers.BatchNormalization(),
            layers.Dense(512, activation="relu"),
            layers.Dropout(rate=0.5),
            layers.BatchNormalization(),
            # layers.Dense(512, activation='relu'),
            # layers.Dropout(rate=0.1),
            # layers.BatchNormalization(),
            # layers.Dense(512, activation='relu'),
            # layers.Dropout(rate=0.1),
            # layers.BatchNormalization(),
            layers.Dense(1, activation="sigmoid"),
        ]
    )
    return model

In [15]:
nn = KerasClassifier(get_model, loss="binary_crossentropy", hidden_layer_dim=100)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=30,
    min_delta=0.001,
    restore_best_weights=True,
)

param_grid = {
    'nn__optimizer':['rmsprop','adam','adagrad'],
    'nn__epochs':[4,8],
    # 'nn__dropout':[0.1,0.2],
    'nn__callbacks': [early_stopping]
}

In [17]:
nn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('nn', nn)
])
nn_pipeline.fit(X_train, y_train)

2022-09-06 18:47:45.264130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-06 18:47:45.264171: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-06 18:47:45.264189: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-0D3E607): /proc/driver/nvidia/version does not exist
2022-09-06 18:47:45.264368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/beaujb/github/faker-classifier/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_

19/19 [==============================] - 1s 4ms/step - loss: 0.8577


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Side', 'teams_region',
                                                   'teammate_role_top',
                                                   'teammate_role_jungle',
                                                   'teammate_role_mid',
                                                   'teammate_role_adc',
                                                   'teammate_role_support',
                                                   'enemy_role_top',
                                                   'enemy_role_jungle',
                                                   'enemy_role_mid',
                                                   'enemy_role_adc',
                                                   'enemy_role_support',
                                                   'tournament_type'])])),
                ('nn',
                 KerasClassifier(hidden_layer_dim=100, loss='binary_crossentropy', model=<function get_model at 0x7f3c745a2790>))])

In [18]:
# pipe = Pipeline([
#     ('scale', OneHotEncoder(handle_unknown="ignore")),
#     ('nn', nn),
# ])
# pipe.fit(X, y)

In [19]:
grid = GridSearchCV(nn_pipeline, cv=3, param_grid=param_grid)

In [20]:
grid.fit(X_train, y_train)

Epoch 1/4


/home/beaujb/github/faker-classifier/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense_3/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense_3/embedding_lookup_sparse/Reshape:0", shape=(None, 125), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense_3/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


13/13 [==============================] - 1s 3ms/step - loss: 0.8953
Epoch 2/4
13/13 [==============================] - 0s 4ms/step - loss: 0.8392
Epoch 3/4
13/13 [==============================] - 0s 4ms/step - loss: 0.6539
Epoch 4/4
7/7 [==============================] - 0s 1ms/step
Epoch 1/4


/home/beaujb/github/faker-classifier/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Reshape:0", shape=(None, 127), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_2/dense_6/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


13/13 [==============================] - 1s 4ms/step - loss: 0.8872
Epoch 2/4
13/13 [==============================] - 0s 4ms/step - loss: 0.7721
Epoch 3/4
13/13 [==============================] - 0s 4ms/step - loss: 0.7044
Epoch 4/4
7/7 [==============================] - 0s 2ms/step
Epoch 1/4


...

/home/beaujb/github/faker-classifier/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_18/dense_54/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_18/dense_54/embedding_lookup_sparse/Reshape:0", shape=(None, 129), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_18/dense_54/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


13/13 [==============================] - 1s 4ms/step - loss: 0.9431
Epoch 2/8
13/13 [==============================] - 0s 3ms/step - loss: 0.9225
Epoch 3/8
13/13 [==============================] - 0s 4ms/step - loss: 0.8169
Epoch 4/8
13/13 [==============================] - 0s 3ms/step - loss: 0.8672
Epoch 5/8
13/13 [==============================] - 0s 4ms/step - loss: 0.8934
Epoch 6/8
13/13 [==============================] - 0s 4ms/step - loss: 0.8042
Epoch 7/8
13/13 [==============================] - 0s 4ms/step - loss: 0.8679
Epoch 8/8
7/7 [==============================] - 0s 1ms/step
Epoch 1/4


/home/beaujb/github/faker-classifier/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_19/dense_57/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_19/dense_57/embedding_lookup_sparse/Reshape:0", shape=(None, 131), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_19/dense_57/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


19/19 [==============================] - 1s 4ms/step - loss: 0.8416
Epoch 2/4
19/19 [==============================] - 0s 4ms/step - loss: 0.7526
Epoch 3/4
19/19 [==============================] - 0s 4ms/step - loss: 0.7626
Epoch 4/4
19/19 [==============================] - 0s 4ms/step - loss: 0.7308


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Side',
                                                                          'teams_region',
                                                                          'teammate_role_top',
                                                                          'teammate_role_jungle',
                                                                          'teammate_role_mid',
                                                                          'teammate_role_adc',
                                                                          'teammate_role_support',
                                                                          'enemy_role_top',
                                                                          'enemy_role_jungle',
                                                                          'enemy_role_mid',
                                                                          'enemy_role_adc',
                                                                          'enemy_role_support',
                                                                          'tournament_type'])])),
                                       ('nn',
                                        KerasClassifier(hidden_layer_dim=100, loss='binary_crossentropy', model=<function get_model at 0x7f3c745a2790>))]),
             param_grid={'nn__callbacks': [<keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>],
                         'nn__epochs': [4, 8],
                         'nn__optimizer': ['rmsprop', 'adam', 'adagrad']})

In [21]:
# summarize results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.681895 using {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 4, 'nn__optimizer': 'rmsprop'}
0.681895 (0.002393) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 4, 'nn__optimizer': 'rmsprop'}
0.681895 (0.002393) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 4, 'nn__optimizer': 'adam'}
0.573604 (0.036838) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 4, 'nn__optimizer': 'adagrad'}
0.680203 (0.000000) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 8, 'nn__optimizer': 'rmsprop'}
0.680203 (0.004145) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 8, 'nn__optimizer': 'adam'}
0.554992 (0.046585) with: {'nn__callbacks': <keras.callbacks.EarlyStopping object at 0x7f3c745c11f0>, 'nn__epochs': 8, 'nn__optim